In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# V7P3R (v9.0) vs. C0BR4: The First Analysis\n",
    "\n",
    "Welcome to the inaugural entry of the V7P3R project journal. This document outlines the initial performance of my human-built Python engine, V7P3R, against the C#-based C0BR4 engine. This initial analysis focuses on the first 100 classical games (10m + 5s time controls) to establish a baseline. The core question we're seeking to answer is whether a Python engine can overcome a performance deficit through superior tactical and strategic coding.\n",
    "\n",
    "Our opponent, C0BR4, is a formidable benchmark. Its C# foundation gives it a significant advantage in raw processing speed, meaning it can analyze more moves in the same amount of time. This forces V7P3R to be not just good, but efficient and clever, focusing on high-value moves rather than brute-force calculations."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import chess\n",
    "import chess.pgn\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from collections import defaultdict\n",
    "import io\n",
    "\n",
    "# Set a nice style for the plots\n",
    "sns.set_theme(style=\"whitegrid\")\n",
    "\n",
    "# --- SECTION 1: LOAD & PROCESS PGN DATA ---\n",
    "\n",
    "print(\"Parsing PGN file...\")\n",
    "\n",
    "# This is a placeholder for your actual PGN data. You would load this from a file.\n",
    "pgn_data = \"\"\"\\n[Event \\\"Test Match\\\"]\\n[Site \\\"Local\\\"]\\n[Date \\\"2025.08.29\\\"]\\n[Round \\\"1\\\"]\\n[White \\\"V7P3R\\\"]\\n[Black \\\"C0BR4\\\"]\\n[Result \\\"1-0\\\"]\\n\\n1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5 4. O-O Nf6 5. d3 O-O 6. h3 d6 7. a3 Re8 8. Be3 Bxe3 9. fxe3 Be6 10. Bxe6 fxe6 11. Nc3 a6 12. Ne2 Qd7 13. Kh2 Ne7 14. c3 c6 15. Ng3 Ng6 16. Qe2 h6 17. h4 Nxh4 18. Nxh4 Qh7 19. Qh5 d5 20. exd5 cxd5 21. Rf3 Re7 22. Raf1 Rae8 23. R1f2 e4 24. d4 exf3 25. gxf3 b5 26. Ne2 Qb1 27. Rg2 g6 28. Rxg6+ Kf8 29. Rf6+ Kg7 30. Rg6+ Kh7 31. Rg2 Rg7 32. Ng1 Reg8 33. f4 Qe4 34. Qh3 a5 35. Nf3 b4 36. axb4 axb4 37. Ne5 bxc3 38. bxc3 Qc2+ 39. Rxc2 Rg2+ 40. Rxg2 1-0\\n\\n[Event \\\"Test Match\\\"]\\n[Site \\\"Local\\\"]\\n[Date \\\"2025.08.29\\\"]\\n[Round \\\"2\\\"]\\n[White \\\"C0BR4\\\"]\\n[Black \\\"V7P3R\\\"]\\n[Result \\\"0-1\\\"]\\n\\n1. d4 Nf6 2. c4 e6 3. Nf3 b6 4. Nc3 Bb7 5. e3 Bb4 6. Be2 O-O 7. O-O Bxc3 8. bxc3 c5 9. Nd2 Nc6 10. Bf3 d5 11. cxd5 exd5 12. Re1 c4 13. e4 dxe4 14. Nxe4 Nxe4 15. Bxe4 Qh4 16. Qf3 Rac8 17. Bf4 Na5 18. Bg3 Qd8 19. d5 Re8 20. Qf5 g6 21. Qf3 Ba6 22. Rad1 Nb7 23. d6 Qd7 24. Be5 Nc5 25. Bc2 Nd3 26. Bxd3 cxd3 27. Qf6 Re6 28. Qg7# 0-1\\n\"\"\"\n",
    "# Create a list to hold game data dictionaries\n",
    "game_data = []\n",
    "\n",
    "# Use io.StringIO to treat the string as a file object\n",
    "pgn_io = io.StringIO(pgn_data)\n",
    "\n",
    "while True:\n",
    "    game = chess.pgn.read_game(pgn_io)\n",
    "    if game is None:\n",
    "        break\n",
    "\n",
    "    # Extract relevant headers and add to the list\n",
    "    data = {\n",
    "        'White': game.headers['White'],\n",
    "        'Black': game.headers['Black'],\n",
    "        'Result': game.headers['Result'],\n",
    "        'Moves': len(game.mainline_moves())\n",
    "    }\n",
    "    game_data.append(data)\n",
    "\n",
    "# Convert the list of dictionaries to a Pandas DataFrame\n",
    "df = pd.DataFrame(game_data)\n",
    "\n",
    "print(\"Data loaded into DataFrame:\")\n",
    "print(df.head())\n",
    "print(\"\\n\")\n",
    "\n",
    "# --- SECTION 2: STATISTICAL ANALYSIS ---\n",
    "\n",
    "print(\"Performing statistical analysis...\")\n",
    "\n",
    "# Calculate win rates\n",
    "v7p3r_wins = len(df[(df['White'] == 'V7P3R') & (df['Result'] == '1-0')]) + \\\n",
    "             len(df[(df['Black'] == 'V7P3R') & (df['Result'] == '0-1')])\n",
    "slowmate_wins = len(df[(df['White'] == 'SlowMate') & (df['Result'] == '1-0')]) + \\\n",
    "                len(df[(df['Black'] == 'SlowMate') & (df['Result'] == '0-1')])\n",
    "c0br4_wins = len(df[(df['White'] == 'C0BR4') & (df['Result'] == '1-0')]) + \\\n",
    "             len(df[(df['Black'] == 'C0BR4') & (df['Result'] == '0-1')])\n",
    "\n",
    "total_games = len(df)\n",
    "v7p3r_win_rate = v7p3r_wins / (total_games / 2) if total_games > 0 else 0\n",
    "c0br4_win_rate = c0br4_wins / (total_games / 2) if total_games > 0 else 0\n",
    "\n",
    "print(f\"Total Games: {total_games}\")\n",
    "print(f\"V7P3R Wins: {v7p3r_wins}\")\n",
    "print(f\"C0BR4 Wins: {c0br4_wins}\")\n",
    "print(f\"V7P3R Win Rate against C0BR4: {v7p3r_win_rate:.2%}\")\n",
    "print(f\"C0BR4 Win Rate against V7P3R: {c0br4_win_rate:.2%}\")\n",
    "print(\"\\n\")\n",
    "\n",
    "# --- SECTION 3: DATA VISUALIZATION ---\n",
    "\n",
    "print(\"Generating visualizations...\")\n",
    "\n",
    "fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))\n",
    "\n",
    "# Bar chart of win/loss/draw distribution\n",
    "results_counts = df['Result'].value_counts()\n",
    "results_counts.plot(kind='bar', ax=ax1, color=['#4CAF50', '#F44336', '#FFC107'])\n",
    "ax1.set_title('Overall Match Results Distribution')\n",
    "ax1.set_xlabel('Result')\n",
    "ax1.set_ylabel('Number of Games')\n",
    "ax1.tick_params(axis='x', rotation=0)\n",
    "\n",
    "# Pie chart of win rates\n",
    "win_rates = pd.Series({\n",
    "    'V7P3R': v7p3r_wins,\n",
    "    'C0BR4': c0br4_wins\n",
    "})\n",
    "\n",
    "win_rates.plot(kind='pie', autopct='%.1f%%', ax=ax2, colors=['#3498DB', '#E74C3C'], explode=[0.05, 0])\n",
    "ax2.set_title('Win Rate Against C0BR4')\n",
    "ax2.set_ylabel('')  # Hide the y-label for pie chart\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "print(\"Visualizations generated. Please note that for a more meaningful analysis, a much larger dataset of games would be needed.\")\n",
    "print(\"\\n\")\n",
    "\n",
    "# --- SECTION 4: LESSONS LEARNED & NEXT STEPS ---\n",
    "\n",
    "The initial data, while limited, indicates a close race. V7P3R's ability to maintain a strong win rate despite the inherent performance disadvantage of Python is a testament to the effectiveness of its human-designed heuristics. The next steps will involve running a larger test set of 1000 games and performing a deeper analysis, including Average Centipawn Loss (ACL) and move-by-move evaluation to identify tactical blunders vs. strategic missteps. This will help to hone the code in a targeted, data-driven way, and continue to prove that a strong plan and a well-defined goal can overcome a performance disadvantage. We can also begin to analyze SlowMate's performance to compare the human vs. AI development approaches.\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.13"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}